In [1]:
import sys, warnings, time, os, copy, gc, re, random, pickle#, cudf
warnings.filterwarnings('ignore')
from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
# pd.set_option('display.max_rows', 50)
# pd.set_option('display.max_columns', None)
# pd.set_option("display.max_colwidth", 10000)
import seaborn as sns
sns.set()
from pandas.io.json import json_normalize
from pprint import pprint
from pathlib import Path
from tqdm import tqdm
tqdm.pandas()
from collections import Counter
from datetime import datetime, timedelta
#import cudf

from sklearn.cluster import KMeans
from sklearn import preprocessing

In [2]:
class Clustering_HandM():
    def customers_preprocessing(self, customers, dropcol=['postal_code'] , **kwargs):
        customers = customers.drop(dropcol, axis=1)
        customers_col = list(customers.columns)
        
        if 'fashion_news_frequency' in customers_col :
            customers['fashion_news_frequency'] = customers['fashion_news_frequency'].replace('NONE','None')
            customers['fashion_news_frequency'] = customers['fashion_news_frequency'].replace({np.nan :0, 'None':0, 'Monthly':1, 'Regularly':2})
            
        if 'club_member_status' in customers_col:
            customers['club_member_status'] = customers['club_member_status'].replace({np.nan :0, 'PRE-CREATE':1, 'ACTIVE':2, 'LEFT CLUB':-1})
            
        if 'age' in customers_col:
            customers['age'] = customers['age'].fillna(-1)
            
        if 'FN' in customers_col:
            customers['FN'] = customers['FN'].fillna(0)

        if 'Active' in customers_col:
            customers['Active'] = customers['Active'].fillna(0)
            
            print(f'###NULL DESCRIPTION###\n{customers.isnull().sum()}')
            
        return customers
    
    
    def clustering(self, df, predcol, usecol, normmethod='StandardScaler', clusters=12, DEBUG=False):
        
        X = np.array(df[usecol])
        
        if normmethod == 'StandardScaler':
            nm = preprocessing.StandardScaler()
            X = nm.fit_transform(X)
        elif normmethod == 'minMax':
            nm = preprocessing.MinMaxScaler()
            X = nm.fit_transform(X)
        print(f'NormarlizationMethod:{normmethod}')
        
        km = KMeans(n_clusters=clusters, random_state=2022)
        km.fit(X)
        print('Distortion: %.2f'% km.inertia_)

        pred = km.labels_
        df_pred = pd.DataFrame(pred, columns=['pred'])
        df_pred = pd.concat([df, df_pred], axis=1)
        
        df_norm = pd.DataFrame(X, columns=usecol)
        print(df_norm.describe())
        
        
        
        if DEBUG:
            df_norm = pd.concat([df[predcol], df_norm], axis=1)
            return df_pred, df_norm
        else:
            return df_pred



In [3]:
DEBUG = False
PATH_INPUT = r'../input/'

In [4]:
customers = pd.read_csv(PATH_INPUT + 'customers.csv')

clst = Clustering_HandM()
customers = clst.customers_preprocessing(customers)

###NULL DESCRIPTION###
customer_id               0
FN                        0
Active                    0
club_member_status        0
fashion_news_frequency    0
age                       0
dtype: int64


In [5]:
customers.head()

,customer_id,FN,Active,club_member_status,fashion_news_frequency,age
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0.0,0.0,2,0,49.0
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0.0,0.0,2,0,25.0
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0.0,0.0,2,0,24.0
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0.0,0.0,2,0,54.0
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,1.0,1.0,2,2,52.0


In [6]:
usecol = ['club_member_status', 'fashion_news_frequency', 'age', 'FN', 'Active']
predcol = ['customer_id']

dfCustomers = clst.clustering(customers, predcol=predcol, usecol=usecol, )

NormarlizationMethod:StandardScaler
Distortion: 215064.47
       club_member_status  fashion_news_frequency           age            FN  \
count        1.371980e+06            1.371980e+06  1.371980e+06  1.371980e+06   
mean         6.477082e-15           -1.226199e-15 -8.864969e-17  3.345628e-15   
std          1.000000e+00            1.000000e+00  1.000000e+00  1.000000e+00   
min         -1.017633e+01           -7.312792e-01 -2.500119e+00 -7.299677e-01   
25%          2.702679e-01           -7.312792e-01 -8.087807e-01 -7.299677e-01   
50%          2.702679e-01           -7.312792e-01 -3.352061e-01 -7.299677e-01   
75%          2.702679e-01            1.368392e+00  8.825574e-01  1.369924e+00   
max          2.702679e-01            1.368392e+00  4.265234e+00  1.369924e+00   

             Active  
count  1.371980e+06  
mean  -4.038759e-15  
std    1.000000e+00  
min   -7.153301e-01  
25%   -7.153301e-01  
50%   -7.153301e-01  
75%    1.397956e+00  
max    1.397956e+00  


In [7]:
dfCustomers.head()

,customer_id,FN,Active,club_member_status,fashion_news_frequency,age,pred
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0.0,0.0,2,0,49.0,9
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0.0,0.0,2,0,25.0,0
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0.0,0.0,2,0,24.0,0
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0.0,0.0,2,0,54.0,3
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,1.0,1.0,2,2,52.0,1
